Imports

In [1]:
#Omid55
import requests
import pandas as pd
import pickle as pk
import csv
import time
import numpy as np

import seaborn as sns
import matplotlib.pylab as plt
% matplotlib inline

In [2]:
URL = {
    'BASE': 'https://{proxy}.api.pvp.net/api/lol/{region}/{rest}',
    'STATIC_BASE': 'https://global.api.pvp.net/api/lol/static-data/{region}/v1.2/{rest}',
    
    'MATCHLIST_URL': 'v2.2/matchlist/by-summoner/{summonerId}?seasons=SEASON{season}',
    'MATCH_URL': 'v2.2/match/{matchId}',
    'CHAMPION_URL': 'champion/{id}?champData=all',
    'SUMMONER_URL': 'v1.4/summoner/{summonerId}',
    #'SUMMONER_WINS_LOSSES_URL': 'v2.5/league/by-summoner/{summonerId}'  # NOT TRUE (VERY LARGE NUMBERS)
    'SUMMONER_WINS_LOSSES_PER_CHAMPION_URL': 'v1.3/stats/by-summoner/{summonerId}/ranked?season=SEASON{season}'
    
    # /api/lol/{region}/v1.3/stats/by-summoner/{summonerId}/ranked: this is for getting the experience 
    #  of player (summonerId) with different champions and also ALL EXPERIENCE one person has
    
    # /api/lol/{region}/v1.3/game/by-summoner/{summonerId}/recent: games that one summoner plays 
    #  with other people
}

REGIONS = {
    'north america': 'na',
    'europe west': 'euw'
}

In [3]:
class Match(object):
    
    def __init__(self):
        self.winners = []
        self.losers = []
        self.duration = -1
        

In [4]:
class RiotAPI(object):
    
    def __init__(self, api_key, region=REGIONS['north america']):
        self.api_key = api_key
        self.region = region
        self.champions = {}
#         self.champions_allinfo = {}
#         self.champions_allinfo_saved = False
        self.summoner_wins_losses = {}
        
    def _request(self, base, rest, params={}):
        #key = base + ',' + rest + ',' + str(params)
        #if key not in self.requests:
        args = {'api_key': self.api_key}
        args.update(params)
        response = requests.get(
            URL[base].format(
                rest=rest,
                proxy=self.region,
                region=self.region,
            ),
            params=args
        )
        #print(response.request.url)
        time.sleep(1.2)
        return response.json()
            #self.requests[key] = response.json()
        #return self.requests[key]
        
    
    def _base_request(self, rest, params={}):
        return self._request('BASE', rest, params)
        
    def _static_request(self, rest, params={}):
        return self._request('STATIC_BASE', rest, params)
    
    # functions
    def get_summoner_level(self, sid):
        rest = URL['SUMMONER_URL'].format(
            summonerId=sid
        )
        return self._base_request(rest)
    
    def _get_list_of_match_ids(self, sid, season):
        rest = URL['MATCHLIST_URL'].format(
            summonerId=sid,
            season=season
        )
        result = self._base_request(rest)
        if 'matches' in result:
            for match in result['matches']:
                yield match['matchId']
                
    def _get_match(self, mid):
        rest = URL['MATCH_URL'].format(
            matchId=mid
        )
        result = self._base_request(rest)
        return result
        
    def get_champion_all_info(self, championId):
        rest = URL['CHAMPION_URL'].format(
            id=championId
        )
        return self._static_request(rest)
    
    def get_summoner_wins_losses(self, sid, season):
#         rest = URL['SUMMONER_WINS_LOSSES_URL'].format(
#             summonerId=sid
#         )
#         result = self._base_request(rest)
#         wins_losses = np.zeros(2)
#         for en in result[str(sid)][0]['entries']:
#             wins_losses += np.array([en['wins'], en['losses']])
#         return wins_losses

        # We can also use matchlist too ...
        if sid not in self.summoner_wins_losses:
            res = {}
            rest = URL['SUMMONER_WINS_LOSSES_PER_CHAMPION_URL'].format(
                summonerId=sid,
                season=season
            )
            result = self._base_request(rest)
            if 'champions' in result:
                for ch in result['champions']:
                    if ch['id']:
                        res[ch['id']] = [ch['stats']['totalSessionsWon'], ch['stats']['totalSessionsLost']]
            self.summoner_wins_losses[sid] = res
        return self.summoner_wins_losses[sid]
        
    
#     def get_champion_index(self, championId):
#         if championId not in self.champions:
#             myid = len(self.champions)
#             self.champions_allinfo[myid] = self.get_champion_all_info(championId)
#             if not self.champions_allinfo_saved and len(self.champions_allinfo) == 132:
#                 with open('DATAChampionsAllInfo.pkl', 'wb') as output:
#                     pk.dump(self.champions_allinfo, output)
#                     self.champions_allinfo_saved = True
#             self.champions[championId] = myid
#         return self.champions[championId]
    
    def get_matches_champions_and_summonerIds_before_game(self, season, just_Ids=True):
        #for sid in range(1,1000000):    #for sid in range(1000000,5000000):
        with open('summonerId_list.pkl', 'rb') as f:
            summoners = pk.load(f)
        for sid in summoners:
            matchids = self._get_list_of_match_ids(sid, season)
            for matchid in matchids:
                match = self._get_match(matchid)
                if 'participants' in match:
                    losers = []
                    winners = []
                    winners_sid = []
                    losers_sid = []
                    
                    for member in match['participants']:
                        suId = [pi['player']['summonerId'] for pi in match['participantIdentities'] if pi['participantId']==member['participantId']]#[0]
                        if member['stats']['winner']:
                            winners_sid += suId
                            
                            if just_Ids:
                                winners += [member['championId']]
                            else:
                                winners += (self.get_champion_capabilities(member['championId']))
                        else:
                            losers_sid += suId
                            
                            if just_Ids:
                                losers += [member['championId']]
                            else:
                                losers += (self.get_champion_capabilities(member['championId']))
                    data = {'matchId': match['matchId'], 'duration': match['matchDuration'], 'champions': winners + losers, 'summoners': winners_sid + losers_sid}
                    yield data
                    
                    
                    
with open('../MyKey2.key', 'r') as key_file:
    KEY = key_file.readline().strip()
api = RiotAPI(KEY)

# Run the script

In [5]:
LIMIT = 1100
champions_data = []
summoners_data = []
dt = api.get_matches_champions_and_summonerIds_before_game(season=2016)

data = []
for d in dt:
    data.append(d)
    if not len(data) % 1000:
        print((len(data)))
    if len(data) > LIMIT:
        break

1000


In [6]:
# unique people
tmp = []
for d in data[:100]:
    tmp += d['summoners']
people = set(tmp)

In [7]:
len(people)

875

In [8]:
sid2indx = {}
def get_index_of_summonerId(sid):
    if sid not in sid2indx:
        sid2indx[sid] = len(sid2indx)
    return sid2indx[sid]

In [20]:
W = np.zeros((len(people),len(people)))
L = np.zeros((len(people),len(people)))
c = 0
for sid in list(people)[:200]:
    c += 1
    if c % 10 == 0:
        print(c)
    matchids = api._get_list_of_match_ids(sid, season=2015)
    for matchid in matchids:
        match = api._get_match(matchid)
        if 'participants' in match:
            sid_index = get_index_of_summonerId(sid)
            for member in match['participants']:
                suId = [pi['player']['summonerId'] for pi in match['participantIdentities'] if pi['participantId']==member['participantId']][0]
                if suId != sid and suId in people:
                    suId_index = get_index_of_summonerId(suId)
                    if member['stats']['winner']:
                        W[sid_index, suId_index] += 1
                    else:
                        L[sid_index, suId_index] += 1
W = np.maximum(W, np.matrix.transpose(W))
L = np.maximum(L, np.matrix.transpose(L))
print('Done.')

KeyboardInterrupt: 